In [1]:
# !pip install dash_bootstrap_components

In [2]:
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import numpy as np

app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

In [3]:
# uncomment this line to load data from local file (last update Nov 5 2021)
# location = 'covid.csv'
location = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
df = pd.read_csv(location)

In [4]:
df['date'] = pd.to_datetime(df.date)

countries = df.location.unique().tolist()
exclude = ['Africa', 'Asia', 'Europe', 'European Union', 'North America', 
           'South America', 'Oceania', 'World', 'International', 'High income',
           'Upper middle income', 'Lower middle income', 'Low income']
countries = list(set(countries).difference(exclude))

continents = ['Africa', 'Asia', 'Europe', 'North America', 'South America', 'Oceania']

countries_df = df[df['location'].isin(countries)]
continent_df = df[df['location'].isin(continents)]

# sort dataframe by date
df_recent = countries_df.sort_values(by='date', ascending=False)

# get most recent date in the dataframe
most_recent = df_recent.date.iloc[0]

# filter the data for the most recent date only
df_recent = df_recent[df_recent['date'] == most_recent]

df_recent = df_recent[df_recent['total_cases'].notna()]

# sort dataframe by date
continent_df_recent = continent_df.sort_values(by='date', ascending=False)

# get most recent date in the dataframe
most_recent = continent_df_recent.date.iloc[1]

# filter the data for the most recent date only
continent_df_recent = continent_df_recent[continent_df_recent['date'] == most_recent]

In [5]:
world = df[df['location'] == 'World']

cards = [
    dbc.Card(
        [
            html.H2("{:,}".format(int(world['total_cases'].iloc[-2])), className="card-title"),
            html.P("Total cases", className="card-text"),
        ],
        body=True,
        color="light",
    ),
    dbc.Card(
        [
            html.H2("{:,}".format(int(world['total_deaths'].iloc[-2])), className="card-title"),
            html.P("Total Deaths", className="card-text"),
        ],
        body=True,
        color="danger",
        inverse=True,
    ),
    dbc.Card(
        [
            html.H2("{:,}".format(int(world['people_fully_vaccinated'].iloc[-2])), className="card-title"),
            html.P("Total Vaccinations", className="card-text"),
        ],
        body=True,
        color="primary",
        inverse=True,
    ),
]

first_row = dbc.Row([dbc.Col(card) for card in cards])

In [6]:
world_map = px.scatter_geo(df_recent, 
                locations="iso_code",
                size="total_cases",
                color='continent',
                hover_name='location',
                hover_data=['total_deaths', 'total_vaccinations'],
                width=1200, height=600,
                title='Map of cases'
            ).update(layout=dict(title=dict(x=0.5)))

second_row = dbc.Row([
    dbc.Col(dcc.Graph(figure=world_map))
])

In [7]:
fig1 = px.line(df[df['location'] == 'World'],
        x='date',
        y='new_cases',
        title="Daily cases"
       ).update(layout=dict(title=dict(x=0.5)))

fig2 = px.bar(continent_df_recent, 
       y='location', 
       x=['total_cases', 'total_deaths'],
        title="Cases and Deaths per Continent",
       orientation='h').update(layout=dict(title=dict(x=0.5)))

third_row = dbc.Row([
    dbc.Col(dcc.Graph(figure=fig1)),
    dbc.Col(dcc.Graph(figure=fig2))
])

In [8]:
fourth_row = dbc.Row([
    dbc.Col(dcc.Graph(id='graph3')),
    dbc.Col(dcc.Graph(id='graph4')),
])

In [9]:
CONTENT_STYLE = {
    "margin-left": "8rem",
    "margin-right": "8rem",
    "padding": "2rem 1rem",
}

app.layout = html.Div([
    
    dbc.Row([html.H1("COVID-19 : "+str(most_recent.date()), style={'textAlign': 'center'})]),
    first_row,
    second_row,
    third_row,
    dcc.Dropdown(
        id='my_dropdown',
        options=[{'label':str(c), 'value': str(c)} for c in countries_df.location.unique()],
        multi=True,
        value=['']
    ),
    fourth_row,
    
    
],
    style =CONTENT_STYLE

)


@app.callback(
    Output(component_id='graph3', component_property='figure'),
    Output(component_id='graph4', component_property='figure'),
    Input(component_id='my_dropdown', component_property='value')
)
def update_my_graph(dropDown_value):
    fig3 = px.scatter(df_recent[(np.isin(df_recent.location, dropDown_value))],
        x='people_fully_vaccinated_per_hundred',
        y='new_deaths_smoothed_per_million',
        hover_name='location',
        size='total_cases_per_million',
        color='location',
        title='Vaccinated & Deaths'
        ).update(layout=dict(title=dict(x=0.5)))

    fig4 = px.line(countries_df[np.isin(countries_df.location, dropDown_value)],
        x='date',
        y='new_deaths_smoothed_per_million',
        hover_name='location',
        color='location',
        title="World's Daily Deaths"
        ).update(layout=dict(title=dict(x=0.5)))
    
    return fig3, fig4



app.run_server(mode='external', port=8060)

Dash app running on http://127.0.0.1:8060/
